In [ ]:
import requests, time
import pandas as pd
import pprint
 
 
import pandas as pd
csv_path = r"data_code.xlsx"
 
df = pd.read_excel(csv_path)
 
codes = df["BSE Exchange Code"].to_list()
print(codes[:10])
 
url = "https://api.bseindia.com/BseIndiaAPI/api/AnnSubCategoryGetData/w"
 
params_template = {
    "pageno": 1,
    "strCat": "Result",
    "strPrevDate": "20250922",
    "strSearch": "P",
    "strToDate": "20251222",
    "strType": "C",
    "subcategory": -1
}
 
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://www.bseindia.com/",
    "Origin": "https://www.bseindia.com",
    "Accept": "application/json, text/javascript, */*; q=0.01",
}
 
session = requests.Session()
all_dfs = []   # collect DataFrames here
 
for idx,code in enumerate(codes[2100:]):
    params = params_template.copy()
    params["strScrip"] = str(code)
 
    response = session.get(url, params=params, headers=headers, verify=False)
    print("Scrip", code, "-> Status:", response.status_code)
 
    if response.status_code == 200:
        data = response.json()
        if "Table" in data and data["Table"]:
            df = pd.DataFrame(data["Table"])
            # Fix attachment URL
            df["ATTACHMENTNAME"] = df["ATTACHMENTNAME"].apply(
                lambda x: "https://www.bseindia.com/xml-data/corpfiling/AttachHis/" + str(x)
            )
            # pprint.pprint(df.head(1).to_dict())  # peek at first row
            all_dfs.append(df)
    else:
        print("❌ Request failed for", code)
   
    time.sleep(2.5)
 
# Concatenate all results into one DataFrame
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv("all_announcements_6.csv", index=False)
    print("✅ Saved", len(final_df), "rows to all_announcements.csv")
else:
    print("⚠️ No data collected")

In [ ]:
import os, re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

download_dir = r"C:\Users\rando\Downloads" 
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# --- CSS selectors for dropdown items ---
state_items_css  = "#j_idt43_items li"
rto_items_css    = "#selectedRto_items li"
yaxis_items_css  = "#yaxisVar_items li"
xaxis_items_css  = "#xaxisVar_items li"
year_items_css   = "#selectedYear_items li"

# --- Helper for safe clicks ---
def safe_click(locator, retries=3, delay=1):
    for attempt in range(retries):
        try:
            el = wait.until(EC.element_to_be_clickable(locator))
            driver.execute_script("arguments[0].click();", el)
            return
        except StaleElementReferenceException:
            print("Stale element, retrying:", locator)
            time.sleep(delay)
    raise Exception(f"Could not click {locator} after {retries} retries")


# --- Select Month ---
def select_month(month_label, retries=3):
    for attempt in range(retries):
        try:
            # Always re-open the dropdown to get a fresh DOM
            safe_click((By.ID, "groupingTable:selectMonth_label"))
            time.sleep(0.4)

            # Re-locate the overlay items each time
            month_items = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#groupingTable\\:selectMonth_items li"))
            )

            # Find the desired month by text
            for m in month_items:
                if m.text.strip() == month_label:
                    driver.execute_script("arguments[0].click();", m)
                    return
        except StaleElementReferenceException:
            print(f"Stale element while selecting {month_label}, retrying...")
            time.sleep(0.5)
    raise Exception(f"Could not select month {month_label} after {retries} retries")


all_dataframes = []

# --- Open States dropdown once to get list ---
safe_click((By.ID, "j_idt43_label")) #"j_idt38_label"
time.sleep(0.5)
states = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, state_items_css)))

for i in range(30, 32):  #len(states) # skip "All States" #do haryana
    safe_click((By.ID, "j_idt43_label"))
    time.sleep(0.4)
    states = driver.find_elements(By.CSS_SELECTOR, state_items_css)

    state = states[i]
    print("\nSTATE:", state.text)
    driver.execute_script("arguments[0].click();", state)
    time.sleep(.5)

    # --- Wait for RTOs ---
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, rto_items_css)) > 1)
    safe_click((By.ID, "selectedRto_label"))
    time.sleep(0.4)
    rtos = driver.find_elements(By.CSS_SELECTOR, rto_items_css)

    for r in rtos[1:]:   # skip "All Offices"
        print("RTO:", r.text)
        driver.execute_script("arguments[0].click();", r)
        time.sleep(.3)

        # --- Set Y-Axis ---
        safe_click((By.ID, "yaxisVar_label"))
        time.sleep(0.3)
        for opt in driver.find_elements(By.CSS_SELECTOR, yaxis_items_css):
            if "Vehicle Category" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set X-Axis = Month Wise ---
        safe_click((By.ID, "xaxisVar_label"))   # open dropdown
        time.sleep(0.4)

        xaxis_opts = driver.find_elements(By.CSS_SELECTOR, "#xaxisVar_items li")
        for opt in xaxis_opts:
            if "Vehicle Class" in opt.text or "Vehicle Class" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Set Year ---
        safe_click((By.ID, "selectedYear_label"))
        time.sleep(0.4)
        for opt in driver.find_elements(By.CSS_SELECTOR, year_items_css):
            if "2025" in opt.text:
                driver.execute_script("arguments[0].click();", opt)
                break

        # --- Click Refresh ---
        safe_click((By.ID, "j_idt85"))
        time.sleep(.5)
        print("\tRefreshed for", state.text, "->", r.text)
        
        
        # --- Select Month ---

        months = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
        for month in months:
            select_month(month)
            time.sleep(0.3)
            # --- Click Excel download ---
            safe_click((By.ID, "groupingTable:xls"))
            time.sleep(.6)

            # files = sorted([os.path.join(download_dir, f) for f in os.listdir(download_dir)], key=os.path.getmtime)
            # path = files[-1]

            # # --- Detect format ---
            # ext = os.path.splitext(path)[1].lower()

            # try:
            #     if ext in [".xls", ".xlsx"]:
            #         # Try Excel first
            #         try:
            #             df = pd.read_excel(path, engine="openpyxl")  # for .xlsx
            #         except Exception:
            #             # Fallback: treat as CSV if Excel parsing fails
            #             df = pd.read_csv(path)
            #     else:
            #         # Default to CSV
            #         df = pd.read_csv(path)

            #     print("Downloaded shape:", df.shape)


                # --- Clean content ---
                # content = df.iloc[2:, 1:]
                # category_class = df.columns[0]
                # rto_clean = re.sub(r"\s+", "", category_class, flags=re.IGNORECASE)
                # content["RTO"] = [rto_clean] * content.shape[0]
                # content["STATE"] = state.text

                # # --- Check empty/non-empty ---
                # if df.shape[1] > 8 and df.shape[0]>3:
                #     print("✅ Non-empty table for", state.text, "->", r.text)
                # else:
                #     print("⚠️ Empty table for", state.text, "->", r.text)

                # all_dataframes.append(content)
            
            # except Exception as e:
            #     print("❌ Failed to parse downloaded file:", e)

# if all_dataframes:
#     master_df = pd.concat(all_dataframes, ignore_index=True)
#     master_df.to_excel("vahan_master_class.xlsx", index=False)
#     print("Saved consolidated data to vahan_master_class.xlsx")
# else:
#     print("No data collected.")

driver.quit()

In [ ]:
import os

folder_path = r"C:\Users\kaustubh.keny\Downloads\kk"

for i, file in enumerate(os.listdir(folder_path), start=1):
    if file.lower().endswith(".xlsx"):
        old_path = os.path.join(folder_path, file)
        new_name = f"report_j{i}.xlsx"   # customize pattern here
        new_path = os.path.join(folder_path, new_name)
        
        os.rename(old_path, new_path)
        print(f"Renamed: {file} -> {new_name}")

In [ ]:
import os, re
import pandas as pd

folder_path = r"C:\Users\rando\Downloads\data-2023"

# collect all dataframes here
global_data = []

regex = r"Vehicle Category Wise Vehicle Class Data\s*of(.*),\s+((?:Andaman & Nicobar Islands|Andhra Pradesh|Arunachal Pradesh|Assam|Bihar|Chhattisgarh|Chandigarh|UT of DNH and DD|Delhi|Goa|Gujarat|Himachal Pradesh|Haryana|Jharkhand|Jammu and Kashmir|Karnataka|Kerala|Ladakh|Lakshadweep|Maharashtra|Meghalaya|Manipur|Madhya Pradesh|Mizoram|Nagaland|Odisha|Punjab|Puducherry|Rajasthan|Sikkim|Tamil Nadu|Tripura|Uttarakhand|Uttar Pradesh|West Bengal))\s+\(([A-Z]+,\d{4})\)$"

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    try:
        ext = os.path.splitext(file_path)[1].lower()
        if ext in [".xls", ".xlsx"]:
            try:
                df = pd.read_excel(file_path, engine="openpyxl", header=None)
            except Exception:
                df = pd.read_csv(file_path, header=None)
        else:
            df = pd.read_csv(file_path, header=None)

        name = str(df.iloc[0, 0])
        content = df.iloc[2:, 1:]  # skip header rows, drop first col
        row, col = content.shape

        if matches := re.findall(regex, name, re.IGNORECASE):
            rto, state, date = matches[0]
            add_data = pd.DataFrame({
                "RTO":   [rto.strip()]   * row,
                "STATE": [state.strip()] * row,
                "DATE":  [date.strip()]  * row
            })
            final_df = pd.concat([add_data, content.reset_index(drop=True)], axis=1)
            global_data.append(final_df)

    except Exception as e:
        print("❌ Failed to parse downloaded file:", file, e)

# combine all into one master dataframe
if global_data:
    master_df = pd.concat(global_data, ignore_index=True)
    master_df.to_csv("VAHAN_DATA_2025.csv", index=False)
    print("✅ Saved consolidated data to VAHA_DATA_2023.csv")
else:
    print("⚠️ No data collected.")

AMFII DISTRIBUTOR DATA

In [ ]:
import pandas as pd
import requests
import os
import time
import warnings
from urllib3.exceptions import InsecureRequestWarning
from datetime import datetime

# Suppress only InsecureRequestWarning
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

def fetch_distributor_data(output_path, type_, cities, page_size=50):
    base_url = "https://www.amfiindia.com/api/distributor-agent"
    throttle = 2  # seconds between requests
    all_records = []

    print(f"Pulling Data for total cities: {len(cities)}.")

    for corp in type_:
        for city in cities:
            print(f"Fetching data for city: {city}, type: {corp}...")
            
            # First request to get pageCount
            params = {
                "strOpt": corp,
                "city": city,
                "search": "",
                "page": 1,
                "pageSize": page_size
            }

            response = requests.get(base_url, params=params, verify=False)
            response.raise_for_status()
            data = response.json()

            # Collect first page
            for rec in data.get("data", []):
                rec["Type"] = corp
                all_records.append(rec)

            # Get total page count
            page_count = data.get("meta", {}).get("pageCount", 1)
            print(f"Total pages for {city} ({corp}): {page_count}")

            # Loop through remaining pages
            for page in range(2, page_count + 1):
                params["page"] = page
                response = requests.get(base_url, params=params, verify=False)
                response.raise_for_status()
                page_data = response.json()
                for rec in page_data.get("data", []):
                    rec["Type"] = corp
                    all_records.append(rec)
                time.sleep(throttle)
                print(f"Data fetched for page {page}/{page_count}")

    # Convert to DataFrame
    df = pd.DataFrame(all_records)

    # Save to single CSV with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = os.path.join(output_path, f"amfi_dist_{timestamp}.csv")
    df.to_csv(output_file, index=False)
    print(f"Saved {len(df)} rows → {output_file}")

if __name__ == "__main__":
    output_folder = r"C:\Users\kaustubh.keny\Downloads\AMFI_Distributor_Data"
    os.makedirs(output_folder, exist_ok=True)

    type_ = ["Individual", "Corporate"]
    
    path = r"cities.csv"
    df = pd.read_csv(path)
    
    cities = df.iloc[:,0].to_list()
    fetch_distributor_data(output_folder, type_, cities[5250:5500]) # do 5000 to 6000
    

AMFII TER DATA

In [ ]:
import requests
import time
import csv
from datetime import datetime

def fetch_data(config):
    response = requests.get(config["url"], params=config["payload"], verify=False)
    response.raise_for_status()
    return response.json()

def save_to_csv(data, filename):
   
    if "data" not in data:
        print("No 'data' field found in response.")
        return
   
    records = data["data"]
    if not records:
        print("No records found.")
        return
   
    headers = list(records[0].keys())
   
    with open(filename, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        writer.writerows(records)
   
    print(f"Data saved to {filename}")

def main():
    print("Fetching data from AMFI API...")
    data = fetch_data(CONFIG)
   
    time.sleep(CONFIG["throttle_seconds"])

    save_to_csv(data, f"{CONFIG["filename"]}_{CONFIG["payload"]["date"]}.csv")


# ---------------- CONFIG ----------------
CONFIG = {
    "url": "https://www.amfiindia.com/api/tracking-difference",
    "payload": {
        "MF_ID": "all",
        "date": "01-May-2022"
    },
    "throttle_seconds": 2,    #delay
    "filename": "tracking_error"
}

if __name__ == "__main__":
    main()

In [ ]:
import requests, time
import pandas as pd
from datetime import datetime, timedelta

def fetch_tracking_error(from_date: str, to_date: str, mf_id: str = "all", output_file: str = "tracking_error.xlsx"):
    """
    Fetch AMFI tracking error data for each day in range and save to one sheet.
    
    Args:
        from_date (str): Start date in format 'dd-mon-yyyy' (e.g. '25-nov-2025')
        to_date (str): End date in format 'dd-mon-yyyy'
        mf_id (str): Mutual Fund ID parameter (default 'all')
        output_file (str): Output Excel file
    """
    # Parse dates
    start = datetime.strptime(from_date, "%d-%b-%Y")
    end = datetime.strptime(to_date, "%d-%b-%Y")

    all_data = []

    #day-loop
    current = start
    while current <= end:
        strdt = current.strftime("%d-%b-%Y").lower()  
        url = f"https://www.amfiindia.com/api/tracking-error-data?MF_ID={mf_id}&strdt={strdt}"
        print(f"Fetching {url} ...")

        try:
            resp = requests.get(url, verify=False)
            resp.raise_for_status()
            data = resp.json()  # API returns JSON

            
            df = pd.DataFrame(data)
            df["date"] = strdt
            df["MF_ID"] = mf_id

            all_data.append(df)
            time.sleep(5)

        except Exception as e:
            print(f"Failed for {strdt}: {e}")

        current += timedelta(days=1)

   
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(output_file, index=False)
        print(f"Saved {len(final_df)} rows to {output_file}")
    else:
        print("No data fetched.")

# Example usage
fetch_tracking_error("01-jun-2022", "31-dec-2022", mf_id="all", output_file="tracking_error_22.xlsx")

In [ ]:
import pandas as pd
import requests
import os
import time


def fetch_ter_data(output_path, months):
    base_url = "https://www.amfiindia.com/api/populate-te-rdata-revised"
    throttle = 1  # seconds between requests

    for month in months:
        print(f"Fetching data for {month}...")
        all_records = []

        # First request to get pageCount
        params = {
            "MF_ID": "All",
            "Month": month,
            "strCat": -1,
            "strType": -1,
            "page": 1,
            "pageSize": 100
        }

        response = requests.get(base_url, params=params, verify=False)
        response.raise_for_status()
        data = response.json()
        all_records.extend(data.get("data", []))


        page_count = data.get("meta", {}).get("pageCount", 1)
        print(f"Total pages for {month}: {page_count}")

        # Loop through remaining pages
        for page in range(2, page_count + 1):
            params["page"] = page
            response = requests.get(base_url, params=params, verify=False)
            response.raise_for_status()
            page_data = response.json()
            all_records.extend(page_data.get("data", []))
            time.sleep(throttle)
            print(f"Run completed for {page}/{page_count}.")


        df = pd.DataFrame(all_records)

        output_file = os.path.join(output_path, f"AMFI_TER_{month}.csv")
        df.to_csv(output_file, index=False)
        print(f"Saved {len(df)} rows for {month} → {output_file}")

if __name__ == "__main__":
    output_folder = r"C:\Users\kaustubh.keny\Downloads\AMFI_TER_Data"
    os.makedirs(output_folder, exist_ok=True)
    
    months = ["10-2021", "11-2021", "12-2021"] #"07-2019", "08-2019", "09-2019", "10-2019", "11-2019", "12-2019"

    fetch_ter_data(output_folder, months)


AMFII FUND PERFORMANCE

In [ ]:
import requests
import pandas as pd
from datetime import datetime

url = "https://www.amfiindia.com/gateway/pollingsebi/api/amfi/fundperformance"

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.amfiindia.com/"
}


report_date = "30-Sep-2025"  


category_map = {
    1: [1, 12],
    2: [13, 29],
    3: [30, 40],
    4: [36, 37],
    5: [38, 39]
}

results = []

for maturity_type in [1, 2]:  # Open Ended, Close Ended
    for category, subcategories in category_map.items():
        for subcat in subcategories:
            payload = {
                "maturityType": maturity_type,
                "category": category,
                "subCategory": subcat,
                "mfid": 0,
                "reportDate": report_date
            }

            response = requests.post(url, json=payload, headers=headers, verify=False)
            if response.status_code == 200:
                data = response.json()
                if data["validationStatus"] == "SUCCESS" and data["data"]:
                    df = pd.DataFrame(data["data"])
                    df["maturityType"] = maturity_type
                    df["category"] = category
                    df["subCategory"] = subcat
                    results.append(df)
                    print(f"✅ Fetched: M{maturity_type} C{category} S{subcat}")
                else:
                    print(f"⚠️ No data: M{maturity_type} C{category} S{subcat}")
            else:
                print(f"❌ Failed: M{maturity_type} C{category} S{subcat} → {response.status_code}")

# Combine all results and save to Excel
if results:
    final_df = pd.concat(results, ignore_index=True)
    filename = f"AMFI_FundPerformance_{report_date.replace('-', '')}.xlsx"
    final_df.to_excel(filename, index=False)
    print(f"\n📁 Data saved to {filename}")
else:
    print("\n🚫 No data retrieved.")

In [ ]:
import requests
import pandas as pd
import time
 
BASE_URL = "https://api.mospi.gov.in/api/plfs/getData"
 
def fetch_plfs_all(params: dict, output_file: str = "plfs_data.csv", sleep_sec: int = 1):
 
    all_data = []
 
    # Start with page 1 to get metadata
    params["page"] = 1
    resp = requests.get(BASE_URL, params=params, verify=False)
    resp.raise_for_status()
    first = resp.json()
 
    if not first.get("statusCode"):
        raise Exception("API returned error: " + str(first))
 
    meta = first.get("meta_data", {})
    total_pages = meta.get("totalPages", 1)
    print(f"Total pages: {total_pages}")
 
 
    if "data" in first:
        all_data.extend(first["data"])
 
    for page in range(2, total_pages + 1):
        params["page"] = page
        print(f"Fetching page {page}/{total_pages} ...")
        try:
            resp = requests.get(BASE_URL, params=params)
            resp.raise_for_status()
            result = resp.json()
            if "data" in result:
                all_data.extend(result["data"])
            time.sleep(3)
        except Exception as e:
            print(f"Failed for page {page}: {e}")
        time.sleep(sleep_sec)
 
    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv(output_file, index=False)
        print(f"Saved {len(df)} rows to {output_file}")
    else:
        print("No data fetched.")
 
if __name__ == "__main__":
    payload = {
        "indicator_code": 8,
        "limit": 20,
        "year": "2017-18,2018-19,2019-20",
        "age_code": "1,2,3,4",
        "education_code": "1,2,3,4,5,6,7,8,9,10",
        "gender_code": "1,2,3",
        "religion_code": "1,2,3,4,5",
        "sector_code": "1,2,3",
        "social_category_code": "1,2,3,4,5",
        "state_code": "1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,99",
        "weekly_status_code": "1,2",
    }
 
    fetch_plfs_all(payload, output_file=f"Average_Gross_2020-24.csv", sleep_sec=0.5)